In [1]:
import time
import nrrd
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import h5py

from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.utils import random_seeds_from_mask
from dipy.reconst.dti import TensorModel
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import actor, window
from dipy.io.image import save_nifti
from dipy.io import read_bvals_bvecs
from dipy.core import gradients
from dipy.tracking.streamline import Streamlines
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, gradient_table_from_bvals_bvecs
from dipy.reconst.dti import fractional_anisotropy

from dipy.tracking import utils

import src.dwi_tools as dwi_tools
import src.nn_helper as nn_helper

Using TensorFlow backend.


In [2]:
pTrainData_fibrePrediction = 'train_prediction_grid_normalized_dti.h5'
pTrainData_fibreTracking = 'train_tracking_grid_normalized_dti.h5'
pTrainInput = 'train_input_normalized_dti_'
noCrossings = 3

# Data Handling

load bvec/bval and compute gradient table

In [3]:
bvals, bvecs = read_bvals_bvecs('100307/bvals', '100307/bvecs')
gtab = gradient_table(bvals=bvals, bvecs=bvecs,b0_threshold=5)

In [4]:
img = nb.load('100307/data.nii.gz')
dwi = img.get_data()
aff = img.affine
img = nb.load('100307/T1w_acpc_dc_restore_1.25.nii.gz')
t1 = img.get_data()
binarymask, options = nrrd.read('100307/nodif_brain_mask.nrrd')

In [5]:
aff = np.eye(4,4)

visualize DWI data

In [ ]:
axial_middle = dwi.shape[2] // 2
plt.figure('Showing the datasets')
plt.subplot(1, 2, 1).set_axis_off()
plt.imshow(dwi[:, :, axial_middle, 0].T, cmap='gray', origin='lower')
plt.subplot(1, 2, 2).set_axis_off()
plt.imshow(dwi[:, :, axial_middle, 10].T, cmap='gray', origin='lower')
plt.show()

# Tractography

In this section we'll generate streamlines using different approaches. This is gonna be the foundation for the evaluation of our method. We'll also employ simulated as well as curated data for our analysis.

In [6]:
# roi_idx = (slice(20, 50), slice(55, 85), slice(38, 39)) #  splenium of the corpus callosum
from dipy.tracking.utils import random_seeds_from_mask, seeds_from_mask
ccmask = np.zeros(binarymask.shape)
ccmask[20:50,55:85,38:39] = 1

In [7]:
ccseeds = seeds_from_mask(ccmask)

In [8]:
ccmask, options = nrrd.read('100307/100307-ccSegmentation.nrrd')
ccseeds = seeds_from_mask(ccmask, affine=aff)
ccseedsNoAffine = seeds_from_mask(ccmask)

Slicer/UKF Data

## Diffusion Tensor Imaging

compute fractional anisotropy and select seeds_count seeds per voxel with FA > 0.3

In [ ]:
import dipy.reconst.dti as dti
dti_wls = dti.TensorModel(gtab)
fit_wls = dti_wls.fit(dwi)
FA = fit_wls.fa
seeds = random_seeds_from_mask(FA > 0.5, seeds_count=1)

In [ ]:
sphere = get_sphere('symmetric724')
start_time = time.time()
dtipeaks = peaks_from_model(model=dti_wls,
                            data=dwi,
                            sphere=sphere,
                            relative_peak_threshold=.5,
                            min_separation_angle=25,
                            mask=binarymask,
                            return_odf=False,
                            parallel=True,
                            normalize_peaks=False,
                            nbr_processes=48
                           )
runtime = time.time() - start_time

In [ ]:
GFA = dtipeaks.gfa
print('Runtime ' + str(runtime) + 's / GFA.shape (%d, %d, %d)' % GFA.shape)

In [ ]:
classifier = ThresholdTissueClassifier(dtipeaks.gfa, .25)
streamlines_generator = LocalTracking(dtipeaks, classifier, ccseeds, np.identity(4), step_size=.1)
#streamlines_generator = LocalTracking(dtipeaks, classifier, ccseeds, aff, step_size=.1)
streamlines = Streamlines(streamlines_generator)
streamlines_filtered = dwi_tools.filterStreamlinesByLength(streamlines, 50)

In [ ]:
dwi_tools.visStreamlines(streamlines_filtered,t1,vol_slice_idx=40)
#dwi_tools.visStreamlines(streamlines_filtered,t1,vol_slice_idx=76)

## Q-ball Reconstruction

In [ ]:
csamodel = CsaOdfModel(gtab, 4)
sphere = get_sphere('symmetric724')
start_time = time.time()
csapeaks = peaks_from_model(model=csamodel,
                            data=dwi,
                            sphere=sphere,
                            relative_peak_threshold=.5,
                            min_separation_angle=25,
                            mask=binarymask,
                            return_odf=False,
                            parallel=True,
                            normalize_peaks=False)

GFA = csapeaks.gfa
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s / GFA.shape (%d, %d, %d)' % GFA.shape)

tracking

In [ ]:
classifier = ThresholdTissueClassifier(dtipeaks.gfa, .25)
streamlines_generator = LocalTracking(csapeaks, classifier, seeds, np.identity(4), step_size=.5)
streamlines = Streamlines(streamlines_generator)
streamlines_filtered = dwi_tools.filterStreamlinesByLength(streamlines, 50)

Visualisation

In [ ]:
dwi_tools.visStreamlines(streamlines,t1)
#dwi_tools.visStreamlines(streamlines_filtered,t1,vol_slice_idx=76)

## Constrained Spherical Deconvolution

Use in case of b = 2,500 – 3,000 s/mm² data

In [ ]:
response, ratio = auto_response(gtab, dwi, roi_radius=10, fa_thr=0.5)
print(response)
print(ratio)
csd_model = ConstrainedSphericalDeconvModel(gtab, response)
sphere = get_sphere('symmetric724')
start_time = time.time()
csd_peaks = peaks_from_model(model=csd_model,
                             data=dwi,
                             sphere=sphere,
                             mask=binarymask,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             parallel=True)
GFA = csd_peaks.gfa
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s / GFA.shape (%d, %d, %d)' % GFA.shape)

In [ ]:
classifier = ThresholdTissueClassifier(csd_peaks.gfa, .25)
streamlines_generator = LocalTracking(csd_peaks, classifier, ccseeds, np.identity(4), step_size=.5)
streamlines = Streamlines(streamlines_generator)
streamlines_filtered = dwi_tools.filterStreamlinesByLength(streamlines, 50)

In [ ]:
streamlines_filtered.shape

Visualisation

In [ ]:
dwi_tools.visStreamlines(streamlines_filtered,t1)

### Store intermediate data

In [ ]:
np.save(pTrainInput + "_sl_filt.npy",streamlines_filtered)
np.save(pTrainInput + "_sl.npy",streamlines)
np.save(pTrainInput + "_seeds.npy",seeds)

# Generate Training data

prepare training data for fibre direction network

In [9]:
dwi_B0normalized = dwi / np.max(dwi[:, :, :, gtab.b0s_mask]) # normalize data.. typically averaging would be used however that would yield dwi values way above 1 (6.5)
dwi_B0normalized = dwi_B0normalized[:, :, :, ~gtab.b0s_mask]  # remove B0 scans from dataset

In [10]:
noX = 8
noY = 8
noZ = 8
coordinateScaling = 0.5

In [ ]:
import importlib
importlib.reload(dwi_tools)
streamlines_filtered = np.asarray(np.load(pTrainInput + "_sl_filt.npy"))
#np.random.shuffle(streamlines_filtered)
import src.dwi_tools as dwi_tools
import src.nn_helper as nn_helper
start_time = time.time()
train_DWI,train_curPosition, train_LikelyFibreDirections, train_NextFibreDirection = dwi_tools.generatePredictionNetworkTrainingDataFromStreamlines(streamlines_filtered, dwi_B0normalized, noX=noX,noY=noY,noZ=noZ,coordinateScaling=coordinateScaling,distToNeighbours=1, noCrossings = noCrossings)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

In [ ]:
with h5py.File(pTrainData_fibrePrediction,"w") as f:
    f.create_dataset('train_DWI',data=train_DWI)
    f.create_dataset('train_curPosition',data=train_curPosition)   
    f.create_dataset('train_LikelyFibreDirections',data=train_LikelyFibreDirections)   
    f.create_dataset('train_NextFibreDirection',data=train_NextFibreDirection)   

## Training

In [16]:
noGPUs = 3
batch_size = 2**8
batch_size -= batch_size % noGPUs # make batch size divisible by no. of GPUs
epochs = 50

In [12]:
f = h5py.File(pTrainData_fibrePrediction, "r")
train_DWI = np.array(f["train_DWI"].value)
train_lastDirection = np.array(f["train_curPosition"].value)
train_LikelyFibreDirections = np.array(f["train_LikelyFibreDirections"].value)
train_NextFibreDirection = np.array(f["train_NextFibreDirection"].value)
f.close()
#train_lastDirection = np.expand_dims(train_lastDirection, axis=1)

In [13]:
cond = train_NextFibreDirection == [0,0,0]

### data normalization

In [14]:
#train_LikelyFibreDirections = (train_LikelyFibreDirections + 1) / 2
#train_lastDirection = (train_lastDirection + 1) / 2
#train_DWI = nn_helper.normalizeDWI(train_DWI)
train_lastDirection_unit = nn_helper.normalizeStreamlineOrientation(train_lastDirection)
train_NextFibreDirection_unit = nn_helper.normalizeStreamlineOrientation(train_NextFibreDirection)

/home/nico/Code/deepFibreTracking/src/nn_helper.py:27: RuntimeWarning: divide by zero encountered in true_divide
  vecs = np.nan_to_num(vecs / vecNorms[:,None])
/home/nico/Code/deepFibreTracking/src/nn_helper.py:27: RuntimeWarning: invalid value encountered in true_divide
  vecs = np.nan_to_num(vecs / vecNorms[:,None])


### MSD Network

In [ ]:
from keras.layers.advanced_activations import LeakyReLU

import importlib
importlib.reload(nn_helper)

In [ ]:
from keras.callbacks import TensorBoard
msd_simple = nn_helper.get_msd_simplified_trackerNetwork(activation_function = LeakyReLU(), inputShapeDWI=(noX, noY, noZ, 270),depth = 3, noGPUs=noGPUs)
msd_simple.fit([train_DWI], [train_NextFibreDirection_unit], batch_size=batch_size, epochs=epochs, verbose=1,validation_split=0.2, callbacks=[])

In [ ]:
from keras.callbacks import TensorBoard
msd_tracker = nn_helper.get_msd_trackerNetwork(activation_function = LeakyReLU(), inputShape1=(noX, noY, noZ, 288),inputShape2=(1,3),depth = 3, noCrossings = noCrossings, noGPUs=noGPUs)
msd_tracker.fit([train_DWI,train_lastDirection_unit], [train_LikelyFibreDirections,train_NextFibreDirection_unit], batch_size=batch_size, epochs=epochs, verbose=1,validation_split=0.2)

### UNET

In [ ]:
import src.nn_helper as nn_helper
importlib.reload(nn_helper)
unet_tracker = nn_helper.get_3Dunet_advancedTracker(activation_function = LeakyReLU(), inputShapeDWI=(noX, noY, noZ, 288),inputShapeStreamline=(1,3),depth = 3, noGPUs=noGPUs)
unet_tracker.fit([train_DWI, train_lastDirection_unit], [train_NextFibreDirection_unit], batch_size=batch_size, epochs=epochs, verbose=1,validation_split=0.2, callbacks=[])

no batch normalisation

In [ ]:
import src.nn_helper as nn_helper
importlib.reload(nn_helper)
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
pModel = "tractography_model_{epoch:02d}-{val_loss:.6f}.h5"
unet_tracker = nn_helper.get_3Dunet_simpleTracker(activation_function = LeakyReLU(), inputShapeDWI=(noX, noY, noZ, 270),depth = 3, noGPUs=noGPUs)
csv_logger = CSVLogger("tractography_log.csv")
checkpoint = ModelCheckpoint(pModel, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list=[checkpoint, csv_logger]
unet_tracker.fit([train_DWI], [train_NextFibreDirection_unit], batch_size=batch_size, epochs=100, verbose=1, validation_split=0.2, callbacks=callbacks_list)

# MLP

In [15]:
from keras import backend as K

def relu_advanced(x):
    return K.relu(x, max_value=1)

In [17]:
import keras
import importlib
import src.nn_helper as nn_helper
importlib.reload(nn_helper)
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
pModel = "tractography_model_{epoch:02d}-{val_loss:.6f}.h5"
#mlp_tracker = nn_helper.get_mlp_simpleTracker(activation_function = LeakyReLU(), inputShapeDWI=(noX, noY, noZ, 270),depth = 3, noGPUs=noGPUs)
mlp_tracker = nn_helper.get_mlp_simpleTracker(features = 512, lr = 1e-4, activation_function = keras.layers.core.Activation(relu_advanced), inputShapeDWI=(noX, noY, noZ, 270),depth = 3, noGPUs=noGPUs)
mlp_tracker.fit([train_DWI], [train_NextFibreDirection_unit], batch_size=batch_size, epochs=100, verbose=1, validation_split=0.2, callbacks=[])

Train on 57936 samples, validate on 14484 samples
Epoch 1/100
57936/57936 [==============================] - 60s 1ms/step - loss: -0.8640 - val_loss: -0.8916
Epoch 2/100
57936/57936 [==============================] - 56s 972us/step - loss: -0.8938 - val_loss: -0.9263
Epoch 3/100
57936/57936 [==============================] - 57s 977us/step - loss: -0.9129 - val_loss: -0.9331
Epoch 4/100
57936/57936 [==============================] - 57s 991us/step - loss: -0.9228 - val_loss: -0.9342
Epoch 5/100
57936/57936 [==============================] - 57s 981us/step - loss: -0.9284 - val_loss: -0.9367
Epoch 6/100
57936/57936 [==============================] - 57s 986us/step - loss: -0.9334 - val_loss: -0.9428
Epoch 7/100
57936/57936 [==============================] - 57s 992us/step - loss: -0.9371 - val_loss: -0.9474
Epoch 8/100
57936/57936 [==============================] - 57s 981us/step - loss: -0.9405 - val_loss: -0.9504
Epoch 9/100
57936/57936 [==============================] - 58s 994us/ste

## Tracking

In [ ]:
np.prod((8,8,8))

In [ ]:
import src.tracking as tracking
importlib.reload(tracking)

In [ ]:
np.min(train_DWI[0,])

In [ ]:
np.max(train_DWI)

In [ ]:
np.std(train_DWI[0,])

# MLP/Simple Network

In [20]:
import src.tracking as tracking
importlib.reload(tracking)
start_time = time.time()
streamlines_mlp_simple = tracking.applySimpleTrackerNetwork(seeds=ccseeds[0:50],data=dwi_B0normalized, model=mlp_tracker, noX=noX, noY=noY, noZ=noZ, dw = 270, stepWidth = 0.1, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

Runtime 72.57033061981201 s 


In [22]:
np.save('streamlines_mlp_simple.npy', streamlines_mlp_simple)

In [ ]:
import src.tracking as tracking
importlib.reload(tracking)
start_time = time.time()
streamlines_mlp_simple = tracking.applySimpleTrackerNetwork(seeds=ccseeds,data=dwi_B0normalized, model=mlp_tracker, noX=noX, noY=noY, noZ=noZ, dw = 270, stepWidth = 0.1, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

In [ ]:
streamlines_mlp_simple[0,5,:]

In [ ]:
streamlines_mlp_simple[0,6,:]

In [21]:
import src.dwi_tools as dwi_tools
importlib.reload(dwi_tools)
dwi_tools.visStreamlines(streamlines_mlp_simple,t1, vol_slice_idx = 70)

/home/nico/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


## MSD/Simple Network

In [ ]:
import src.tracking as tracking
importlib.reload(tracking)
start_time = time.time()
streamlines_msd_simple = tracking.applySimpleTrackerNetwork(seeds=[ccseeds[0:100]],data=dwi_B0normalized, model=msd_simple, noX=noX, noY=noY, noZ=noZ, dw = 270, stepWidth = 0.1, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

In [ ]:
streamlines_msd_simple[0,5,:]

In [ ]:
streamlines_msd_simple[0,6,:]

In [ ]:
import src.dwi_tools as dwi_tools
importlib.reload(dwi_tools)
dwi_tools.visStreamlines(streamlines_msd_simple,t1, vol_slice_idx = 73)

### MSD/Advanced Tracking Network

In [ ]:
start_time = time.time()
streamlines_msd_adv = tracking.applyTrackerNetwork(seeds=ccseeds,data=dwi, model=msd, noX=noX, noY=noY, noZ=noZ, coordinateScaling=coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

In [ ]:
dwi_tools.visStreamlines(streamlines_msd_adv,t1, vol_slice_idx = 73)

### UNET/Advanced Tracking Network

In [ ]:
start_time = time.time()
streamlines = tracking.applyTrackerNetwork(seeds=ccseeds,data=dwi, model=unet_tracker, noX=noX, noY=noY, noZ=noZ, coordinateScaling=coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

In [ ]:
dwi_tools.visStreamlines(streamlines,t1, vol_slice_idx = 73)